In [1]:
# Reference: https://stackoverflow.com/questions/15514593/importerror-no-module-named-when-trying-to-run-python-script/15622021#15622021
import sys
sys.path.append(r'G:\Grid_Orientation\LISFLOOD_FP\Modelling\Modules')

In [2]:
from randomisation import random_transformation     # For collecting transformation values
from lidar import download_lidar                    # For downloading LiDAR data
from transformation import get_url, \
                            center_calculation       # For getting url file for transformation command and for center calculation
from demReference import padding_combination, \
                         dem_raster_reference       # For padding calculation and dem reference generation
from execution import *                             # For executing transformed and un-transformed functions

import numpy as np

import rioxarray as rxr
import xarray as xr
import geopandas as gpd
import json

from geofabrics import processor

import shutil
import os
import pathlib
import shapely.geometry

# DEMO

In [3]:
resolution_func = 10
lidar_dataset_name = "Wellington_2013"
chunk_size_func = 100
processor_func = 4

h_crs = 2193
v_crs = 7839

result_folder = "dem_bathy4"

data_dir = pathlib.Path(os.getcwd()) / pathlib.Path(fr"S:\Example\geofabrics_DEM\{result_folder}")
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [4]:
# Padding box coordinates
padding_func = [1767345.0, 5469590.0, 1774065.0, 5476310.0]

# Bounding box - Part 1
x0 = padding_func[0]
y0 = padding_func[1]
x1 = padding_func[2]
y1 = padding_func[3]
catchment = shapely.geometry.Polygon([(x0, y0), (x1, y0), (x1, y1), (x0, y1)])
catchment = gpd.GeoSeries([catchment])
catchment = catchment.set_crs(h_crs)

# Write out to file
catchment.to_file(fr"S:\Example\geofabrics_DEM\{result_folder}\catchment_boundary.geojson", crs="EPSG:2193", driver="GeoJSON")

#### Set up for instructions

In [5]:
instruction_json = {}

In [7]:
# # DRAIN
# instruction_json['drains'] = {
#         # drain - output
#         "output": {
#             "crs": {
#                 "horizontal": h_crs,
#                 "vertical": v_crs
#             },
#             "grid_params": {
#                 "resolution": 1
#             }
#         },
        
#         # drain - processing
#         "processing": {
#             "chunk_size": 1000,
#             "number_of_cores": 4
#         },
        
#         # drain - data_paths
#         "data_paths": {
#             "local_cache": fr"S:\Example\geofabrics_DEM",
#             "subfolder": fr"{result_folder}",
#             "catchment_boundary": "catchment_boundary.geojson"
#         },
        
#         # drain - apis
#         "apis": {
#             "open_topography": {
#                 "Wellington_2013": {
#                     "crs": {
#                         "horizontal": 2193,
#                         "vertical": 7839
#                     }
#                 }
#             },
#             "linz": {
#                 "key": "b561fb1517554b1088f89de6eacdbcdb",
#                 "land": {
#                     "layers": [51559]
#                 }
#             }
#         },
        
#         # drain - general
#         "general": {
#             "lidar_classifications_to_keep": [2, 9]
#         },
        
#         # drain - drains
#         "drains": {
#             "width": 5
#         }
        
#     }

In [6]:
instruction_json['rivers'] = {
        # river - output
        "output": {
            "crs": {
                "horizontal": h_crs,
                "vertical": v_crs
            },
            "grid_params":{
                "resolution": 1
            }
        },
        
        # river - processing
        "processing": {
            "chunk_size": 1000,
            "number_of_cores": 4
        },
        
        # river - data_paths
        "data_paths": {
            "local_cache": fr"S:\Example\geofabrics_DEM",
            "subfolder": fr"{result_folder}",
        },
        
        # river - apis
        "apis": {
            "open_topography": {
                "Wellington_2013": {
                    "crs": {
                        "horizontal": 2193,
                        "vertical": 7839
                    }
                }
            },
            "linz": {
                "key": "b561fb1517554b1088f89de6eacdbcdb",
                "land": {
                    "layers": [51559]                                              # SHOULD I ROTATE?
                },
                "bathymetry_contours": {
                    "layers": [50554]                                              # SHOULD I ROTATE?
                }
            }
        },
        
        # river - general
        "general": {
            "set_dem_shoreline": True,
            "bathymetry_contours_z_label": "valdco",
            "drop_offshore_lidar": True,
            "lidar_classifications_to_keep": [2, 9],
            "interpolate_missing_values": False
        },
        
        # river - rivers
        "rivers": {
            "osm_id": 132793862,
            "veg_lidar_classifications_to_keep": [2, 3, 4, 5, 9],
            "max_channel_width": 120, # DON'T NEED TO BE VARIED
            "min_channel_width": 10, # DON'T NEED TO BE VARIED
            "max_bank_height": 2, # NEED VARYING. RANGE TO VARY?
            "rec_alignment_tolerance": 65, # DON'T NEED TO BE VARIED
            "width_centre_smoothing": 10,
            "channel_area_threshold": 141000000, # CAN CHANGE TO 100000000
            "channel_rec_id": 9253579,
            "cross_section_spacing": 10,
            "min_bank_height": 0.75,
            "rec_file": fr"S:\Example\geofabrics_DEM\{result_folder}\rec2_3.geojson",
            "flow_file": fr"S:\Example\geofabrics_DEM\{result_folder}\flow_and_friction.csv.gz"
        }
}

In [23]:
instruction_json['dem'] = {
    # outputs
    "output": {
        "crs": {
            "horizontal": h_crs,
            "vertical": v_crs
        },
        "grid_params":{
            "resolution": resolution_func
        }
    },
    
    # apis
    "apis": {
        "open_topography": {
            f"{lidar_dataset_name}": {
                "crs": {
                    "horizontal": h_crs,
                    "vertical": v_crs
                }
            }
        },
        "linz": {
            "key": "b561fb1517554b1088f89de6eacdbcdb",
            "land": {
                "layers": [51559]                                                        # SHOULD I ROTATE?
            },
        }
    },
    
    # processing
    "processing": {
        "chunk_size": chunk_size_func,
        "number_of_cores": processor_func
    },
    
    # data paths
    "data_paths": {
        "local_cache": fr"S:\Example\geofabrics_DEM",
        "subfolder": fr"{result_folder}",
        "catchment_boundary": "catchment_boundary.geojson",
        # For bathymetry
        "river_bathymetry": ["river_bathymetry.geojson", "fan_bathymetry.geojson"],                    # SHOULD I ROTATE?
        "river_polygons": ["river_polygon.geojson", "fan_polygon.geojson"],                      # SHOULD I ROTATE?
        # Result
        "result_dem": "generated_dem.nc"
    },
    
    # general
    "general": {
        # For lidar
        "set_dem_shoreline": True,
        "drop_offshore_lidar": False,
        "lidar_classification_to_keep": [2],
        "interpolation_method": "nearest",
        "lidar_interpolation_method": "idw",
        
        # For bathymetry
        "bathymetry_contours_z_label": "valdco",                                        # HOW TO CHECK?
        "bathymetry_points_type": ['rivers', 'rivers'],                                 # HOW TO CHECK? OPTIONS?
        "bathymetry_points_z_label": ['bed_elevation_Rupp_and_Smart', "depths"]         # HOW TO CHECK? OPTIONS?
    }
}

In [24]:
with open(fr"S:\Example\geofabrics_DEM\results\instructions.json", "w") as instruction:
    json.dump(instruction_json, instruction)

In [25]:
%%time
runner = processor.RiverBathymetryGenerator(instruction_json['rivers'])
runner.run()
# runner = processor.DrainBathymetryGenerator(instruction_json['drains'])
# runner.run()
runner = processor.RawLidarDemGenerator(instruction_json['dem'])
runner.run()
runner = processor.HydrologicDemGenerator(instruction_json['dem'])
runner.run()

Check files in dataset Wellington_2013
<Client: 'tcp://127.0.0.1:51233' processes=4 threads=4, memory=63.69 GiB>


C:\Program Files\Anaconda3\envs\geofabrics45\lib\site-packages\geofabrics\geometry.py:528: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  points = points.append(points_i)
C:\Program Files\Anaconda3\envs\geofabrics45\lib\site-packages\geofabrics\geometry.py:531: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  polygon = polygon.append(polygon_i)


AttributeError: 'GeoDataFrame' object has no attribute 'to_list'